<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watsonx.governance: Model Deployment and Monitoring (Simplified)

This notebook deploys a German Credit Risk model to Watson Machine Learning (WML), onboards it to Watson OpenScale (WOS), configures monitors (quality, fairness, driftv2, explainability), and injects historical scoring and feedback data.

> **Runtime Requirement**: IBM Runtime 24.1 (Python 3.11 XS)  
> **Services Required**: Watson Machine Learning, Watson OpenScale, Cloud Object Storage  
> **Optional**: Db2 Warehouse or Databases for PostgreSQL for external datamart

Ensure all required credentials and configuration values are defined before running the cell below.


### Package installation


In [ ]:
#If you are executing this notebook in non IBM Watson Studio env then uncomment the pip install statements
# !pip install --upgrade pandas==0.25.3 
# !pip install --upgrade numpy==1.20.3 
# !pip install --upgrade SciPy 
# !pip install --upgrade scikit-learn==1.3.2 
# !pip install --upgrade requests==2.32.2 
# !pip install --upgrade ibm-watson-machine-learning

!pip install --upgrade ibm-watson-openscale

!rm german_credit_data_biased_training.csv
!rm german_credit_feedback_data.json
!rm german_credit_scoring_data.json

!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feedback_data.json -O german_credit_feedback_data.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_scoring_data.json -O german_credit_scoring_data.json


## Step 1: Define Credentials and Basic Configuration




## Configuration Dictionary Inputs

The following table describes the inputs expected in the `config_dict` used to deploy and monitor a model. Some inputs are required, while others are optional and will default to predefined values if not provided.

| Field | Type | Required | Default | Description |
|-------|------|----------|---------|-------------|
| `wml_space_id` | `str` (UUID) | Yes | – | ID of the Watson Machine Learning deployment space. |
| `operational_space_id` | `str` | No | `"production"` | Allowed values: `"production"`, `"pre_production"`. |
| `cloud_api_key` | `str` | Yes | – | IBM Cloud API key used for authentication. |
| `wml_url` | `str` (URL) | No | – | URL of the Watson Machine Learning service. |
| `bucket_name` | `str` | Yes | – | Name of the Cloud Object Storage bucket. |
| `cos_resource_crn` | `str` | Yes | – | Cloud Object Storage resource CRN. |
| `cos_endpoint` | `str` (URL) | Yes | – | Endpoint URL for the COS instance. |
| `db_credentials` | `dict` or `None` | No | `None` | External database credentials, if applicable. |
| `schema_name` | `str` or `None` | No | `None` | Schema name for external database. if applicable. |
| `training_data_df` | `tuple (X, y)` or Spark DataFrame | Yes | – | training_data_df: Tuple (X, y) for sklearn/XGBoost or Spark DataFrame. X must have named columns. Used to infer model schema during publishing to WML.|
| `training_data_file_name` | `str` | Yes | – | Name of the training data file to upload to COS. |
| `model_object` | object | Yes | – | Trained model or pipeline (e.g., scikit-learn, Spark MLlib). |
| `service_provider_name` | `str` | No | `"Watson Machine Learning V1"` | Name of the model provider. |
| `service_provider_description` | `str` | No | `"Added by tutorial WOS notebook."` | Description of the model provider. |
| `model_name` | `str` | No | `"Scikit German Risk Model WML V1"` | Name of the model asset. |
| `deployment_name` | `str` | No | `"Scikit German Risk Deployment WML V1"` | Name of the model deployment. |
| `asset_name` | `str` | No | `"Scikit German Risk Model WML V1"` | Name of the model asset in OpenScale. |
| `label_field` | `str` | Yes | – | Target column in the dataset. |
| `prediction_field` | `str` | Yes | – | Output field for predictions. |
| `probability_fields` | `list[str]` | Yes | – | List of probability output fields. |
| `feature_fields` | `list[str]` | Yes | – | Input features used by the model. |
| `categorical_fields` | `list[str]` | Yes | – | Categorical input features. |
| `feedback_data_file` | `str` | Yes | – | File containing historical feedback records. |
| `scoring_data_file` | `str` | Yes | – | File containing historical scoring requests. |
| `quality_monitor_params` | `dict` | No | If monitor parameters are not provided, they will not be configured and will be skipped. | Parameters for quality monitor. |
| `quality_monitor_thr` | `list[dict]` | No | – | Threshold for triggering quality alerts. |
| `fairness_monitor_params` | `dict` | No | If monitor parameters are not provided, they will not be configured and will be skipped. | Configuration for fairness monitor. |
| `fairness_monitor_thr` | `None` or `list[dict]` | No | – | Threshold for fairness alerts, if applicable. |
| `explainability_params` | `dict` | No | If monitor parameters are not provided, they will not be configured and will be skipped. | Enable or disable explainability. |
| `driftv2_params` | `dict` | No | If monitor parameters are not provided, they will not be configured and will be skipped. | Drift monitor configuration. |
| `problem_type` | `str` | No | `"binary"` | Type of machine learning problem. Examples: `"binary"`, `"regression"`. `"multiclass"` | 
| `input_data_type` | `str` | No | `"structured"` | Type of input data. Examples: `"structured"`, `"unstructured_image"`. |
| `monitor_new_version` | `bool` | No | `False` | If set to `True`, the system will delete and recreate the existing model deployment, service provider, and OpenScale subscription. This is useful when you want to reset everything and start. |
| `wos_service_instance_id` | `str` | No | `None` | The ID of the Watson OpenScale instance. |
| `model_type` | `str` | Yes | `"scikit-learn_1.3"` | WML model type identifier. Must match the framework used. For spark model use `"mllib_3.4"` |
| `software_spec_name` | `str` | No | `"runtime-24.1-py3.11"` | Runtime environment for the model. Must be compatible with the model type. For Spark models use `"spark-mllib_3.4"` |
| `pipeline` | Spark ML Pipeline | Yes (for Spark models) | – |     The original Spark pipeline object used for training. Required only when registering Spark MLlib models to WML. Not applicable for sklearn |
| `wos_service_url` | `str` | No | `None` |     WOS service url  |



Reference: Optional Parameter Configuration

For detailed steps on how to set and configure the optional parameters, please refer to the following notebook:

[Watson OpenScale and Watson ML Engine - scikit-learn Binary Classification Example](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/notebooks/binary/scikit_learn/Watson%20OpenScale%20and%20Watson%20ML%20Engine.ipynb)





In [ ]:
WML_URL = "https://us-south.ml.cloud.ibm.com"
CLOUD_API_KEY = "<EDIT THIS>"
SPACE_ID = "<EDIT THIS>"


TRAINING_DATA_FILE_NAME = "<EDIT THIS>" #eg "german_credit_data_biased_training.csv
COS_RESOURCE_CRN = "<EDIT THIS>" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "<EDIT THIS>" # eg"https://s3.us-south.cloud-object-storage.appdomain.cloud"
BUCKET_NAME = "<EDIT THIS>"


## Step 2: Load and Upload Training Data

This section performs two key tasks:

1. **Load the training dataset** from a local CSV file into a Pandas DataFrame.
2. **Upload the dataset to Cloud Object Storage (COS)** so it can be referenced during model deployment and monitoring.


Ensure that the COS credentials and bucket name are correctly defined before running this step.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import ibm_boto3
from ibm_botocore.client import Config


training_data_df = pd.read_csv(TRAINING_DATA_FILE_NAME)

cos_client = ibm_boto3.resource(
    "s3",
    ibm_api_key_id=CLOUD_API_KEY,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

with open(TRAINING_DATA_FILE_NAME, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, TRAINING_DATA_FILE_NAME).upload_fileobj(Fileobj=file_data)


## Step 3: Train and Evaluate the Model

This section prepares and trains a machine learning pipeline using the German Credit Risk dataset. It also evaluates the model's performance using AUC (Area Under the ROC Curve).


In [4]:

training_data_df = pd.read_csv(TRAINING_DATA_FILE_NAME)
train_data, test_data = train_test_split(training_data_df, test_size=0.2)
features_idx = np.s_[0:-1]
first_record_idx = np.s_[0]
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]

ct = ColumnTransformer([
    ("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))
])

clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([
    ('ct', ct), 
    ('clf_linear', clf_linear)
])


# Fit the scikit-learn pipeline on training data.
# The model must be trained before it can be published to WML.
# Assign the trained model to MODEL_OBJECT for use in the config.
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)
MODEL_OBJECT = risk_model

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction == 'No Risk' else 1 for prediction in predictions]
real_observations = test_data.Risk.replace('Risk', 1).replace('No Risk', 0).values
auc = roc_auc_score(real_observations, indexed_preds)
print("Model AUC:", auc)




# For scikit-learn models, TRAINING_DATA_DF should be a tuple (X, y):
# - X_train must be a pandas DataFrame with named feature columns
# - y_train should be a Series or array with target values
X_train = train_data.drop(columns=["Risk"])
y_train = train_data["Risk"]

TRAINING_DATA_DF = (X_train, y_train)

Model AUC: 0.7496026180458157


/var/folders/86/j1xsn76129g6wdwmpx5x3wp00000gn/T/ipykernel_55770/732796524.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  real_observations = test_data.Risk.replace('Risk', 1).replace('No Risk', 0).values


In [5]:

WML_URL = "https://us-south.ml.cloud.ibm.com"

LABEL_FIELD = "Risk"
PREDICTION_FIELD = "prediction"
PROBABILITY_FIELDS = ["probability"]


FEATURE_FIELDS = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
CATEGORICAL_FIELDS = [
    "CheckingStatus",
    "CreditHistory",
    "LoanPurpose",
    "ExistingSavings",
    "EmploymentDuration",
    "Sex",
    "OthersOnLoan",
    "OwnsProperty",
    "InstallmentPlans",
    "Housing",
    "Job",
    "Telephone",
    "ForeignWorker",
]

SAMPLE_DATA_FIELDS = FEATURE_FIELDS
SAMPLE_DATA_VALUES = [
    [
        "no_checking",
        13,
        "credits_paid_to_date",
        "car_new",
        1343,
        "100_to_500",
        "1_to_4",
        2,
        "female",
        "none",
        3,
        "savings_insurance",
        46,
        "none",
        "own",
        2,
        "skilled",
        1,
        "none",
        "yes",
    ],
    [
        "no_checking",
        24,
        "prior_payments_delayed",
        "furniture",
        4567,
        "500_to_1000",
        "1_to_4",
        4,
        "male",
        "none",
        4,
        "savings_insurance",
        36,
        "none",
        "free",
        2,
        "management_self-employed",
        1,
        "none",
        "yes",
    ],
]

FEEDBACK_DATA_FILE = "german_credit_feedback_data.json"
SCORING_DATA_FILE = "german_credit_scoring_data.json"

QUALITY_MONITOR_PARAMS = {"min_feedback_data_size": 50}
QUALITY_MONITOR_THR = [
    {"metric_id": "area_under_roc", "type": "lower_limit", "value": 0.80}
]
FAIRNESS_MONITOR_PARAMS = {
    "features": [
        {
            "feature": "Sex",
            "majority": ["male"],
            "minority": ["female"],
            "threshold": 0.95,
        },
        {
            "feature": "Age",
            "majority": [[26, 75]],
            "minority": [[18, 25]],
            "threshold": 0.95,
        },
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100,
}
FAIRNESS_MONITOR_THR = None
EXPLAINABILITY_PARAMS = {"enabled": True}
DRIFTV2_FEATURE_IMPORTANCE = {
    "CheckingStatus": 0.027799999999999936,
    "LoanDuration": 0.0,
    "CreditHistory": 0.005449999999999977,
    "LoanPurpose": 0.0023499999999999632,
    "LoanAmount": 0.0,
    "ExistingSavings": 0.0007500000000000284,
    "EmploymentDuration": 0.04099999999999997,
    "InstallmentPercent": 0.0,
    "Sex": 0.014049999999999962,
    "OthersOnLoan": 0.01044999999999996,
    "CurrentResidenceDuration": 0.0,
    "OwnsProperty": 0.03159999999999994,
    "Age": 0.0,
    "InstallmentPlans": 0.0023999999999999577,
    "Housing": 0.0007000000000000117,
    "ExistingCreditsCount": 0.0,
    "Job": 0.003449999999999953,
    "Dependents": 0.0,
    "Telephone": 0.005949999999999967,
    "ForeignWorker": 0.0005499999999999394,
}
DRIFTV2_PARAMS = {
    "min_samples": 10,
    "max_samples": 1000,
    "train_archive": True,
    "features": {"fields": FEATURE_FIELDS, "importances": DRIFTV2_FEATURE_IMPORTANCE},
}


## Step 4: Configure and Run Deployment & Monitoring

This section defines a single configuration dictionary (`config_dict`) that consolidates all required inputs for deploying the model, onboarding it to Watson OpenScale, and enabling monitors.


Once the configuration is complete, the `deploy_and_monitor_model()` function is called to:
1. Deploy the model to Watson Machine Learning.
2. Onboard it to Watson OpenScale.
3. Configure the selected monitors.
4. Inject historical scoring and feedback data.
5. Run the monitors.



In [6]:

config_dict = {
    "wml_space_id": SPACE_ID,
    "wml_url": WML_URL,
    "cloud_api_key": CLOUD_API_KEY,
    "bucket_name": BUCKET_NAME,
    "cos_resource_crn": COS_RESOURCE_CRN,
    "cos_endpoint": COS_ENDPOINT,
    "training_data_file_name": TRAINING_DATA_FILE_NAME,
    "label_field": LABEL_FIELD,
    "prediction_field": PREDICTION_FIELD,
    "probability_fields": PROBABILITY_FIELDS,
    "feature_fields": FEATURE_FIELDS,
    "categorical_fields": CATEGORICAL_FIELDS,
    "feedback_data_file": FEEDBACK_DATA_FILE,
    "scoring_data_file": SCORING_DATA_FILE,
    "monitor_new_version" :True,
    "wml_config": {
        "model_object": MODEL_OBJECT,
        "training_data_df": TRAINING_DATA_DF
    },
}



from ibm_watson_openscale.notebooks_simplification.predictive_models import PredictiveModelDeploymentManager
manager = PredictiveModelDeploymentManager(config=config_dict)
manager.deploy_and_monitor()


Warning - Missing or invalid monitor field: 'quality_monitor_params'
Warning - Missing or invalid monitor field: 'fairness_monitor_params'
Warning - Missing or invalid monitor field: 'explainability_params'
Warning - Missing or invalid monitor field: 'driftv2_params'
Connection to WML established. WML version = 1.0.368
Deleting deployment id: 27c7ab5a-7474-4b56-ba3d-aa5436d03de5
Deleted deployment_id: 27c7ab5a-7474-4b56-ba3d-aa5436d03de5, model_id: 06d957e0-77c7-4a87-a192-646829031e1a
Creating connections...
SUCCESS
Storing model...
Storing model completed successfully; model uid = eb45a498-dc95-4d1b-b576-4890e7ae03f4
Deploying model...


#######################################################################################

Synchronous deployment creation for uid: 'eb45a498-dc95-4d1b-b576-4890e7ae03f4' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in 

WOS_AUTO_CREATED,This data-mart was automatically created by Watson X.Gov on behalf of users for prompt evaluations.,True,active,2025-06-23 12:26:42.542000+00:00,f40df02e-0542-4ea5-9930-83ac604951e0


Using existing datamart f40df02e-0542-4ea5-9930-83ac604951e0
Configuring service provider
Found existing WML service provider 'Watson Machine Learning V1' with ID: 0198374c-e384-7498-9b8f-2fbe540a8edb for the space id 6385487b-5496-49ac-bf85-87e5e29d1ab5
Deleting service provider with ID: 0198374c-e384-7498-9b8f-2fbe540a8edb
Adding new service provider



 Waiting for end of adding service provider 01983751-5b15-75a5-8e4a-088172224c5a 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------


Added new service provider successfully; Service provider ID 01983751-5b15-75a5-8e4a-088172224c5a



 Waiting for end of adding subscription 01983751-7f9e-77f3-b3cd-bca5d055ddfb 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


New subscription created successfully with ID: 01983751-7f9e-77f3-b3cd-bca5d0